In [2]:
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

Определяем теги у точек

In [4]:
#загрузка слоев
def load_layers(folder_path, service_tags):
    service_layers = {}
    for tag in service_tags:
        file_path = os.path.join(folder_path, f"{tag}.geojson")  # Формируем путь к файлу
        if os.path.exists(file_path):  # Проверяем, существует ли файл
            try:
                # Загружаем слой в GeoDataFrame
                gdf = gpd.read_file(file_path)

                # Добавляем столбец с названием тега
                gdf['SERVICE'] = tag

                # Сохраняем только геометрию и название сервиса
                gdf = gdf[["geometry", "SERVICE"]]

                # Сохраняем слой в словарь
                service_layers[tag] = gdf
                print(f"Загружен и обновлен слой: {tag}")
            except Exception as e:
                print(f"Ошибка при загрузке или обработке слоя {tag}: {e}")
        else:
            print(f"Файл для тега '{tag}' не найден.")
    return service_layers

In [6]:
# Папка, где находятся слои
folder_path = "./data/serv"

In [7]:
MED_layers = load_layers(folder_path, MEDICAL_FACILITIES_tags)

Загружен и обновлен слой: child_hospital
Загружен и обновлен слой: child_polyclinic
Загружен и обновлен слой: dentist


In [8]:
MED_layers 

{'child_hospital':                      geometry         SERVICE
 0   POINT (30.26735 59.94232)  child_hospital
 1   POINT (30.33768 60.00438)  child_hospital
 2   POINT (30.28141 59.94687)  child_hospital
 3   POINT (30.37815 59.71048)  child_hospital
 4   POINT (30.18588 59.83767)  child_hospital
 5   POINT (30.26079 59.93432)  child_hospital
 6   POINT (30.41807 59.83872)  child_hospital
 7   POINT (30.32563 60.01711)  child_hospital
 8   POINT (30.33061 59.98933)  child_hospital
 9   POINT (30.59565 59.74856)  child_hospital
 10  POINT (30.36433 59.93441)  child_hospital
 11  POINT (30.36767 59.93241)  child_hospital
 12  POINT (30.34314 59.98067)  child_hospital
 13  POINT (30.29147 59.92516)  child_hospital
 14  POINT (30.36757 59.95403)  child_hospital
 15  POINT (30.36757 59.95403)  child_hospital
 16  POINT (30.31760 59.97185)  child_hospital
 17  POINT (30.32595 59.96602)  child_hospital
 18  POINT (30.41618 59.88245)  child_hospital
 19  POINT (30.31165 59.97371)  child_hosp

In [9]:
def merge_layers(service_layers):
    # Создаем список слоев, которые будем преобразовывать в нужную систему координат
    transformed_layers = []

    for layer in service_layers.values():
        # Проверка, имеет ли слой CRS
        if layer.crs is None:
            raise ValueError("Система координат не установлена в одном из слоев.")

        # Если CRS не в EPSG:3857, выполняем преобразование
        if layer.crs.to_epsg() != 3857:
            layer = layer.to_crs(epsg=3857)
            print(f"Система координат слоя была переведена в EPSG:3857.")

        # Добавляем преобразованный слой в список
        transformed_layers.append(layer)

    # Объединяем все слои в один DataFrame
    all_layers = pd.concat(transformed_layers, ignore_index=True)
    
    # Оставляем только столбцы 'geometry' и 'SERVICE'
    all_layers = all_layers[['geometry', 'SERVICE']]
    
    # Преобразуем в GeoDataFrame с корректной геометрией
    all_layers = gpd.GeoDataFrame(all_layers, geometry='geometry')

    return all_layers

In [10]:
MED_gdf = merge_layers(MED_layers)

Система координат слоя была переведена в EPSG:3857.
Система координат слоя была переведена в EPSG:3857.
Система координат слоя была переведена в EPSG:3857.


In [11]:
MED_gdf = MED_gdf.to_crs(epsg=3857)

In [13]:
MED_gdf.head(1)

,geometry,SERVICE
0,POINT (3369346.324 8386906.143),child_hospital


In [163]:
MED_gdf.explore()

Создаем буфер у каждого сервиса

In [14]:
def build_buffer_zone(gdf, service_radius_dict):
    gdf['buffer'] = gdf.apply(
    lambda row: row['geometry'].buffer(service_radius_dict.get(row['SERVICE'])), axis=1)
    return gdf

In [15]:
MED_buffer = build_buffer_zone(MED_gdf, MEDICAL_FACILITIES_tags)

In [166]:
print(MED_buffer.crs)

EPSG:3857


In [16]:
MED_buffer.head()

,geometry,SERVICE,buffer
0,POINT (3369346.324 8386906.143),child_hospital,"POLYGON ((3369846.324 8386906.143, 3369843.916..."
1,POINT (3377174.755 8400712.222),child_hospital,"POLYGON ((3377674.755 8400712.222, 3377672.348..."
2,POINT (3370911.698 8387919.457),child_hospital,"POLYGON ((3371411.698 8387919.457, 3371409.291..."
3,POINT (3381680.542 8335560.029),child_hospital,"POLYGON ((3382180.542 8335560.029, 3382178.135..."
4,POINT (3360276.457 8363686.129),child_hospital,"POLYGON ((3360776.457 8363686.129, 3360774.049..."


Пересечение сервисов и кварталов

In [22]:
#загружаем геометрию кварталов
blocks_gdf = gpd.read_file("./data/blocks_relation_tagger.geojson")

In [23]:
blocks_gdf = blocks_gdf.to_crs(epsg=3857)

In [24]:
def create_service_table(blocks_gdf, service_tags):
    # Добавляем столбцы для каждого сервиса с начальными значениями 0
    for service in service_tags.keys():
        blocks_gdf[service] = 0

    # Добавляем столбцы для каждого сервиса с начальными значениями 0
    for service in service_tags.keys():
        blocks_gdf['SERVICE'] = 0

    return blocks_gdf

In [25]:
MED_blocks_gdf = create_service_table(blocks_gdf, MEDICAL_FACILITIES_tags)
MED_blocks_gdf.head(1)

,id,geometry,child_hospital,child_polyclinic,dentist,SERVICE
0,0,"POLYGON ((3385731.840 8387954.044, 3385731.840...",0,0,0,0


In [26]:
print(MED_blocks_gdf.crs)

EPSG:3857


In [27]:
def check_intersection(blocks_gdf, buffer_gdf, service_tags):
    for idx, block in tqdm(blocks_gdf.iterrows(), total=blocks_gdf.shape[0], desc="Processing blocks"):
        for service in service_tags.keys():
            # Получаем буферы для текущего сервиса
            relevant_buffers = buffer_gdf.loc[buffer_gdf['SERVICE'] == service, 'geometry']
            
            if not relevant_buffers.empty:
                for buffer_geom in relevant_buffers:
                    intersection = block['geometry'].intersection(buffer_geom)
                    # Проверяем, пересекается ли хотя бы 10% буфера
                    if intersection.area >= 0.1 * buffer_geom.area:
                        blocks_gdf.at[idx, service] = 1
    
    # Устанавливаем столбец 'SERVICE' для блоков, которые пересекаются с хотя бы одним сервисом
    blocks_gdf['SERVICE'] = blocks_gdf.apply(
        lambda row: 1 if any(row[service] == 1 for service in service_tags.keys()) else 0,
        axis=1
    )
    
    return blocks_gdf

In [28]:
MED_blocks_service_gdf = check_intersection(MED_blocks_gdf, MED_buffer, MEDICAL_FACILITIES_tags)

Processing blocks: 100%|██████████| 3839/3839 [02:40<00:00, 23.86it/s]


In [29]:
MED_blocks_service_gdf.head(1)

,id,geometry,child_hospital,child_polyclinic,dentist,SERVICE
0,0,"POLYGON ((3385731.840 8387954.044, 3385731.840...",1,1,1,1
